In [12]:
import numpy as np
import pandas as pd
import xpress as xp
import math

In [13]:
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

In [14]:
rooms_df = pd.read_excel("Room_Data_TAOR[19].xlsx")
R_r = set(rooms_df["ROOM NAME"])
M_r = {row["ROOM NAME"]: row["CAP"] for _, row in rooms_df.iterrows()}  # Room capacities
P_r = {row["ROOM NAME"]: row["SUIT 4 - ROOM LAYOUT"] for _, row in rooms_df.iterrows()}  # Room types
T = set(range(1, 46))
#W = set(range(9, 21)) #Accoridng to Data

# Create a mapping from each timeslot to the day of the week (1=Monday, 2=Tuesday, …, 5=Friday)
day_for_t = {t: ((t - 1) // 9) + 1 for t in sorted(T)}
A_t = [t for t in T if day_for_t[t] == 3 and ((t - 1) % 9) >= 4]

F_day = { 1: 9, 2: 18, 3: 27, 4: 36, 5: 45 }
P_r = {
    row["ROOM NAME"]: "*Workshop" if row["SUIT 4 - ROOM LAYOUT"] in 
    ["4. Layout - Classroom Style", "4. Layout - Boardroom Style", "Empty"] 
    else "*Lecture" if row["SUIT 4 - ROOM LAYOUT"] == "4. Layout - Theatre Style"
    else row["SUIT 4 - ROOM LAYOUT"]
    for _, row in rooms_df.iterrows()
} # Change P_r to match with class requirements


In [15]:
studentdata = pd.read_excel("student_data[10].xlsx")
studentdata["UNN"] = studentdata["UNN"].fillna("").astype(str)

# Group students by course and create a dictionary

G_cs = {course: set(group["UNN"]) for course, group in studentdata.groupby("Course Code")}

# Get the set of unique courses
C_c = set(G_cs.keys())

S_s = set(studentdata["UNN"])


In [16]:
timetable_data = pd.read_excel("School of Mathematics - Timetable Data_first_semester.xlsx")


timetable_data["Activity Type Name"] = timetable_data["Activity Type Name"].fillna("").astype(str)

# Remove duplicates for 'Workshop' and 'Computer Workshop' activities per course
#timetable_data_filtered = timetable_data[~timetable_data.duplicated(subset=["Course Code"], keep="first") | ~timetable_data["Activity Type Name"].isin(["*Workshop", "Computer Workshop"])]
# First filter for rows where Activity Type Name is "*Lecture" or "*Workshop"
filtered_data = timetable_data[timetable_data["Activity Type Name"].isin(["*Lecture", "*Workshop"])]

# Then remove duplicates based on both "Course Code" and "Activity Type Name"
timetable_data_filtered = filtered_data.drop_duplicates(subset=["Course Code", "Activity Type Name"], keep="first")

# Create the set of classes, generating a unique key using an incremental counter per Course Code
unique_keys = {}
E_e = []

for _, row in timetable_data_filtered.iterrows():
    course, activity = row["Course Code"], row["Activity Type Name"]
    unique_keys[course] = unique_keys.get(course, 0) + 1  # Increment counter for this course
    unique_key = f"{course}_{unique_keys[course]}"  # Create a unique key
    E_e.append((course, activity, unique_key))  # Store as tuple

# Create dictionaries V_ew of teaching weeks and D_e of durations
timetable_data_filtered["Unique Key"] = [entry[2] for entry in E_e]  # Add Unique Key column
old_V_ew = timetable_data_filtered.set_index("Unique Key")["Teaching Week Pattern"].to_dict()
D_e = timetable_data_filtered.set_index("Unique Key")["Duration"].to_dict()

D_e_numeric = {}
for key, value in D_e.items():
    if isinstance(value, str):
        parts = value.split(':')
        if len(parts) == 2:
            hours = float(parts[0])
            minutes = float(parts[1])
            D_e_numeric[key] = hours + minutes / 60.0
        else:
            # If the string is not in "HH:MM" format, try converting directly
            D_e_numeric[key] = float(value)
    else:
        D_e_numeric[key] = float(value)

# Now compute the number of time slots (each slot is one hour)
#num_slots = {e: math.ceil(D_e_numeric[e]) for e in D_e_numeric}
num_slots = {e: max(1, math.ceil(D_e_numeric[e])) for e in D_e_numeric}


Q_e = {row["Course Code"]: row["Allocated Location Name"] for _, row in timetable_data .iterrows()}  # Required room types
E_c = {c: [] for c in C_c}

# Populate E_c by iterating over all classes in E_e.
for e in E_e:
    # Access the course from the tuple. Adjust the index if the course attribute is stored in a different position.
    course = e[0]  
    if course in E_c:
        E_c[course].append(e)
    else:
        # Optionally, handle courses not initially in C_c
        E_c[course] = [e]

/var/folders/mn/x3m1qqts4rn3j7zc2dx5y1ym0000gp/T/ipykernel_12783/3254472516.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timetable_data_filtered["Unique Key"] = [entry[2] for entry in E_e]  # Add Unique Key column


In [17]:
def parse_timeslot_string(slot_str):
    """
    Given a string with timeslots (with commas and ranges),
    return a list of integers.
    """
    times = []
    # Ensure the slot_str is a string (unless it's NaN)
    if not isinstance(slot_str, str):
        if isinstance(slot_str, float) and math.isnan(slot_str):
            return times  # Return an empty list for NaN values
        else:
            slot_str = str(slot_str)
    
    # Split the string by commas
    for token in slot_str.split(','):
        token = token.strip()  # Remove extra whitespace
        if '-' in token:  # If token represents a range, e.g., "26-30"
            start, end = token.split('-')
            # Convert start and end to integers and expand the range
            times.extend(range(int(start), int(end) + 1))
        else:  # If token is a single number
            times.append(int(token))
    return times
 

# Build your parameter V_ew: For each class e, create a list (or set) of weeks w
V_ew = {}
for e, slot_str in old_V_ew.items():
    week_list = parse_timeslot_string(slot_str)
    V_ew[e] = week_list  # If you prefer, convert to a set: set(week_list)
    

In [24]:
V_ew = {e: [w for w in week_list if 9 <= w <= 13] for e, week_list in V_ew.items()}


In [28]:
rooms_df["SUIT 3 - PRAM"] = rooms_df["SUIT 3 - PRAM"].fillna("").astype(str)
high_weight = 100
default_weight = 50  # Default weight for all rooms
math_weight = 1  # Lower weight for rooms in the School of Mathematics


K_r = {}

for _, row in rooms_df.iterrows():
    room = row["ROOM NAME"]
    school = row["SUIT 3 - PRAM"]

    if 'Mathematics' in school:
        K_r[room] = math_weight
    elif 'empty' in school:
        K_r[room] = default_weight
    else:
        K_r[room] = high_weight




In [30]:
prob = xp.problem(name='myproblem')

In [32]:


x = {(e, w, t, r): xp.var(name=f"x_{e}_{w}_{t}_{r}", vartype=xp.binary)
     for e in E_e for w in V_ew.get(e[2], set()) for t in T for r in R_r}


z = {r: xp.var(name=f"z_{r}", vartype=xp.binary) for r in R_r}

y = {(s, t): xp.var(name=f"y_{s}_{t}", vartype=xp.integer, lb=0) for s in S_s for t in T}


y_penalty = {(s, t): xp.var(name=f"y_penalty_{s}_{t}", vartype=xp.integer, lb=0) for s in S_s for t in T}

u = {(e, t): xp.var(vartype=xp.binary, name=f"u_{e}_{t}") for e in E_e for t in T}
# For each event e, for every room in R_r
l = {(e, r): xp.var(vartype=xp.binary, name=f"l_{e}_{r}") for e in E_e for r in R_r}

#h = { e: xp.var(name=f"h_{e}", vartype=xp.integer, lb=1, ub=5) for e in E_e }


v1 = xp.var(name="v1", vartype=xp.integer, lb=0)  
v2 = xp.var(name="v2", vartype=xp.integer, lb=0)  
#u = {(e, t): xp.var(vartype=xp.binary, name=f"u_{e}_{t}") for e in E_e for t in T}

prob.addVariable(list(x.values()))       # Decision variable x
prob.addVariable(list(z.values()))       # Room assignment variables
prob.addVariable(list(y.values()))       # Student timetable variables
prob.addVariable(list(y_penalty.values()))  # Auxiliary penalty variables
prob.addVariable(list(u.values()))
prob.addVariable(list(l.values()))
prob.addVariable([v1, v2])  
#prob.addVariable(list(h.values()))  

In [34]:
# Assuming you have a mapping T_index such that:


timetable_clash_constraints = [
    y[(s, t)] == xp.Sum(
        x[(e, w, t_prime, r)]
        for c in C_c if s in G_cs[c]
        for e in E_c[c]
        for w in V_ew.get(e[2], set())
        for t_prime in range(max(t - int(D_e_numeric[e[2]]) + 1, 1), t + 1)
        for r in R_r
        if (e, w, t_prime, r) in x
    )
    for s in S_s
    for t in sorted(T)
]
prob.addConstraint(timetable_clash_constraints)


In [36]:


room_type_penalty = xp.Sum(
    x[(e, w, t, r)]
    for e in E_e
    for w in V_ew.get(e[2], set())
    for t in T
    for r in R_r
    if (e, w, t, r) in x and e[1] != P_r.get(r, None)
)

avoid_wed_penalty = xp.Sum(
    x[(e, w, t, r)]
    for e in E_e
    for w in V_ew.get(e[2], set())   # allowed weeks for event e
    for r in R_r
    for t in A_t                   # t in Wednesday afternoon
    if (e, w, t, r) in x
)



In [38]:


unique_time = [
    xp.Sum(u[(e, t)] for t in T ) == 1
    for e in E_e
]


unique_room = [
    xp.Sum(l[(e, r)] for r in R_r ) == 1
    for e in E_e
]

prob.addConstraint(unique_time)
prob.addConstraint(unique_room)


In [40]:


room_linking_constraints = []
time_linking_constraints = []

for e in E_e:
    # Get the allowed weeks for event e (using its unique key e[2])
    allowed_weeks = sorted(V_ew.get(e[2], []))
    if allowed_weeks:  # Only if there are allowed weeks
        for w in allowed_weeks:
            # Force room linking: for each room r, sum of x over times in week w equals l[(e, r)]
            for r in R_r:
                expr_room = xp.Sum(x[(e, w, t, r)] for t in T if (e, w, t, r) in x)
                room_linking_constraints.append(expr_room == l[(e, r)])
            # Force time linking: for each timeslot t, sum of x over rooms in week w equals u[(e, t)]
            for t in T:
                expr_time = xp.Sum(x[(e, w, t, r)] for r in R_r if (e, w, t, r) in x)
                time_linking_constraints.append(expr_time == u[(e, t)])

prob.addConstraint(room_linking_constraints)
prob.addConstraint(time_linking_constraints)



In [42]:
room_usage_constraints = [
    xp.Sum(x[(e, w, t, r)] 
    for e in E_e 
    for w in V_ew.get(e[2], set())
    if (e, w, t, r) in x) <= 1
    for t in T
    for r in R_r
]
prob.addConstraint(room_usage_constraints)
#INITIALLY W

In [44]:
linking_xz = [
    x[(e, w, t, r)] <= z[(r)]
    for e in E_e for w in V_ew.get(e[2], set())for r in R_r for t in T  
    if (e, w, t, r) in x
]

prob.addConstraint(linking_xz)

In [46]:
'''no_cross_day_constraints = [
    x[(e, w, t, r)] == 0
    for e in E_e for w in V_ew.get(e[2], set()) for r in R_r for t in T
    for t_prime in range(max(9, t + num_slots[e[2]]-1))
    #for t_prime in range(t, min(t + num_slots[e[2]] , max(T) + 1)) # Duration check
    if t_prime in F_t and (e, w, t, r) in x  # If t' crosses a day boundary, set x to 0
]

prob.addConstraint(no_cross_day_constraints)'''

no_cross_day_constraints = [
    # For every possible assignment, if the event would extend past the day end,
    # force the decision variable to zero.
    x[(e, w, t, r)] == 0
    for e in E_e
    for w in V_ew.get(e[2], set())   # allowed weeks for event e
    for t in T
    for r in R_r
    if (e, w, t, r) in x and (t + num_slots[e[2]] - 1 > F_day[ day_for_t[t] ])
]

prob.addConstraint(no_cross_day_constraints)




In [48]:

'''room_overbooking_constraints = [
    xp.Sum(
        x[(e, w, t_prime, r)]
        for e in E_e 
        if w in V_ew.get(e[2], set())  # Only consider e if week w is allowed for e
        for t_prime in range(t, min(t + num_slots[e[2]], max(T) + 1))
        if (e, w, t_prime, r) in x
    ) <= 1
    for r in R_r
    
    for t in sorted(T)
]
prob.addConstraint(room_overbooking_constraints)'''


# Let T_min be the minimum time slot in T:
T_min = min(T)

no_overlap_constraints = [
    xp.Sum(
        x[(e, w, t_prime, r)]
        for e in E_e
        for t_prime in range(max(T_min, t - int(num_slots[e[2]]) + 1), t + 1)
        if (e, w, t_prime, r) in x
    ) <= 1
    for r in R_r
    for w in  V_ew.get(e[2], set()) #if you want to restrict to allowed weeks
    for t in sorted(T)
]
prob.addConstraint(no_overlap_constraints)


In [50]:
never_overlap_constraints = [
    xp.Sum(
        x[(e, w, t, r)]
        for e in E_e
        if w in V_ew.get(e[2], set()) and (e, w, t, r) in x
    ) <= 1
    for r in R_r
    for w in V_ew.get(e[2], set())
    for t in T
]

prob.addConstraint(never_overlap_constraints)


In [52]:
G_c_sizes = {course: len(students) for course, students in G_cs.items()}


In [54]:
room_capacity_constraints = [
    # For each room r, timeslot t, course c, class e in course c, and week w:
    x[(e, w, t, r)] * G_c_sizes[c] <= M_r[r]
    for r in R_r
    for t in T
    for c in C_c if c in G_cs #and c in E_c  # Only consider courses with enrollment and defined classes
    for e in E_c[c]
    for w in V_ew.get(e[2], set())
    if (e, w, t, r) in x  # Only add constraint if the variable exists
]

# Add constraints to the Xpress model:
prob.addConstraint(room_capacity_constraints)


In [56]:

class_scheduling_constraints = [
    xp.Sum(
        x[(e, w, t, r)]
        for t in T
        for r in R_r
        if (e, w, t, r) in x
    ) == 1
    for e in E_e
    for w in V_ew.get(e[2], set())
]
prob.addConstraint(class_scheduling_constraints)





penalty_constraint = [
                y_penalty[s,t] >= y[s,t] - 1
                for t in T
                 for s in S_s

]
prob.addConstraint(penalty_constraint)

In [58]:
# For each course c, find its events (using the first element of the event tuple)
no_same_course_overlap = []
for c in C_c:
    # Get all events for course c. (Assuming e[0] is the course code.)
    events_for_c = [e for e in E_e if e[0] == c]
    # For each pair of distinct events from the same course:
    for i in range(len(events_for_c)):
        for j in range(i+1, len(events_for_c)):
            e1 = events_for_c[i]
            e2 = events_for_c[j]
            # Determine the common allowed weeks (if each event has its own allowed weeks)
            allowed_weeks = set(V_ew.get(e1[2], set())) & set(V_ew.get(e2[2], set()))
            for w in allowed_weeks:
                for t in T:
                    # Sum over rooms for e1 and e2 at week w, time t must be at most 1.
                    constraint_expr = xp.Sum(x[(e1, w, t, r)] for r in R_r if (e1, w, t, r) in x) + \
                                      xp.Sum(x[(e2, w, t, r)] for r in R_r if (e2, w, t, r) in x)
                    no_same_course_overlap.append(constraint_expr <= 1)
# Add these constraints to the model:
prob.addConstraint(no_same_course_overlap)


In [60]:
w1 = 100  # Weight for student timetable clashes
w2 = 50  # Weight for room assignments
w3 = 50  # Weight for soft penalties
w4 = 50
# First term: Penalize timetable clashes
over_scheduling_penalty = xp.Sum(
    w1 * y_penalty[(s, t)] for s in S_s for t in T
)

# Second term: Penalize room usage (weighted by K_r)
room_penalty = xp.Sum(
    w2 * K_r.get(r, 1) * z[r] for r in R_r if r in z  # Use default weight 1 if missing
)

# Third term: Soft penalties (v1 + v2)
weds_penalty = w3 * (avoid_wed_penalty) 
room_usage_penalty = w4 * (room_type_penalty) #initially v1 and v2

# Define objective function
objective_function = over_scheduling_penalty + room_penalty + weds_penalty + room_usage_penalty

# Set the objective in the model
prob.setObjective(objective_function, sense=xp.minimize)

In [62]:
prob.setControl('miprelstop', 0.2)
#prob.setControl({'feastol': 0.001, 'presolve': 0})

prob.solve()
xp.setOutputEnabled(False)

print("Status:", prob.getProbStatus())
print("Objective:", prob.getObjVal())


FICO Xpress v9.4.2, Hyper, solve started 23:43:49, Mar 31, 2025
Heap usage: 3905MB (peak 3905MB, 200MB system)
Minimizing MILP myproblem using up to 8 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
MIPRELSTOP = .2
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
   2128647 rows      1101905 cols    114691087 elements   1101905 entities
Presolved problem has:
     57913 rows       520673 cols      2880856 elements    520673 entities
LP relaxation tightened
Presolve finished in 702 seconds
Heap usage: 2970MB (peak 8733MB, 200MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  3.41e+02] / [ 2.50e-01,  1.25e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.00e+02] / [ 2.50e-01,  8.90e+01]
  Objective      [min,max] : [ 5.00e+01,  5.00e+03] / [ 5.00e+01,  2.34e+04]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage

SolverError: ?1081 Error: The Xpress-Optimizer license has been lost; this may be because of a connection failure with the license server.

In [ ]:
# Build an ordered list of variables that you added:
all_vars = list(x.values()) + list(z.values()) + list(y.values()) + \
           list(y_penalty.values()) + list(u.values()) + list(l.values())

ordered_x = list(x.items())
solution_values = prob.getSolution()
solution_dict = { key: val for (key, var), val in zip(ordered_x, solution_values) }

solution_rows = []
for key, val in solution_dict.items():
    if val > 0.5:  # or use a threshold appropriate for your model
        solution_rows.append(list(key) + [val])

df_solution = pd.DataFrame(solution_rows, columns=["Event", "Week", "Timeslot", "Room", "Value"])
print(df_solution)
df_solution.to_csv("solution_week5.csv", index=False)
#print("Solution written to solution.csv")


In [32]:
sol_dict = {var.name: val for var, val in zip(all_vars, solution_values)}
y_solution_rows = []
for (s, t), var in y.items():
    # Lookup the variable solution using its name:
    sol_val = sol_dict.get(var.name, 0)
    # Use a threshold for binary/integer variables as needed:
    if sol_val > 0.5:
        y_solution_rows.append([s, t, sol_val])

df_y = pd.DataFrame(y_solution_rows, columns=["Student", "Timeslot", "y_value"])
print(df_y)
df_y.to_csv("y_solution_week1.csv", index=False)
#print("Exported y-solution to y_solution.csv")


Empty DataFrame
Columns: [Student, Timeslot, y_value]
Index: []


In [33]:
sol_dict = {var.name: val for var, val in zip(all_vars, solution_values)}
ypenalty_solution_rows = []
for (s, t), var in y_penalty.items():
    # Lookup the variable solution using its name:
    sol_val = sol_dict.get(var.name, 0)
    # Use a threshold for binary/integer variables as needed:
    if sol_val > 0.5:
        ypenalty_solution_rows.append([s, t, sol_val])

df_penalty = pd.DataFrame(ypenalty_solution_rows, columns=["Student", "Timeslot", "y_value"])
print(df_penalty)
df_y.to_csv("ypenalty_solution_week1.csv", index=False)
#print("Exported y-solution to y_solution.csv")


Empty DataFrame
Columns: [Student, Timeslot, y_value]
Index: []


In [34]:
z_solution_rows = []
for r, var in z.items():
    sol_val = sol_dict.get(var.name, 0)
    if sol_val > 0.5:
        z_solution_rows.append([r, sol_val])

df_z = pd.DataFrame(z_solution_rows, columns=["Room", "z_value"])
print(df_z)
df_z.to_csv("z_solution_week1.csv", index=False)
#print("Exported z-solution to z_solution.csv")


                     Room  z_value
0               JCMB_6206      1.0
1               JCMB_2901      1.0
2  JCMB_Lecture Theatre B      1.0
